# Analysis - zBrains outputs at 3T and 7T in epilepsy
1. zBrain/wBrain (surface)
    a. Histograms of vertex wise scores
        i. sub-comparisons with different smoothing kernels
    b. Quantifying extreme vertex groups
        i. number of identified abnormal areas
        ii. size of each abnormal area (number of adjacent extreme vertices)
2. Brainstats (surface)
    a. t-scores for 3T and 7T
    b. cohen's D map between 3T and 7T images


## 1. Histograms

In [15]:
import os, sys
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import vrtx
import plots

sys.path.append('/host/verges/tank/data/daniel/')  # Replace with the path to Utils
from Utils import gen


In [23]:
# print HHMM_DD-MMM-YYYY
date = dt.datetime.now().strftime('%d%b%Y_%H%M')
#print(f'Running on {date}')

Running on 24Mar2025_2048


In [6]:
import importlib

importlib.reload(vrtx)
importlib.reload(gen)
importlib.reload(plots)

<module 'plots' from '/host/verges/tank/data/daniel/3T7T/z/code/analyses/plots.py'>

In [4]:
# define directories
output_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs"
values_dir = "values"
processed_output_dir = "/host/verges/tank/data/daniel/3T7T/z/outputs/fig_stats"


# 3T-7T ID correspondence
correps_IDs = {
    "path": "/host/verges/tank/data/daniel/3T7T/z/data/pt/IDs_ses_analyses_12Mar.csv",
    "3T_ID": "3T_ID",
    "7T_ID": "7T_ID",
    "3T_SES": "3T_SES",
    "7T_SES": "7T_SES"
}

#id_corresp = pd.read_csv(corresp_ID)

# Study names
MICs = {"name": "MICs"}

PNI = {"name": "PNI"}

#studies = ["MICs", "PNI"]

# zBrain analysis regions
cortex = {
    "region": "cortex",
    "surfaces": ["midthickness", "white"],
    "resolution": "32k",
    "features": ["ADC", "T1map", "volume"], # (list) features to extract
    #"smoothing": [10]
    "smoothing": [2,5,10]
}

hippocampus = {
    "region": "hippocampus",
    "surfaces": ["midthickness"],
    "resolution": "0p5mm",
    "features": ["ADC", "T1map", "volume"], # (list) features to extract
    #"smoothing": [5]
    "smoothing": [1,2,5]
}

subcortex = {
    "region": "subcortex",
    "features": ["ADC", "T1map", "volume"],
    "smoothing": [2,5,10]
}

regions = [cortex, hippocampus, subcortex]

In [8]:
# get list of corresponding 3T, 7T aggregate files
files_lst = plots.corresp_paths(regions, MICs, PNI, output_dir, values_dir)
#print(files_lst)
shape = gen.lstOlst_shape(files_lst,print=False)
print(f"raw shape of files_lst (num files, num studies): {shape}")

# get missing files
missing = plots.get_missingPths(files_lst)

# remove missing files from list
for m in missing:
    files_lst.remove(m)

shape = gen.lstOlst_shape(files_lst,print=False)
print(f"shape of files_lst (num files, num studies): {shape}")


raw shape of files_lst (num files, num studies): 57 by 2
[missingPth] Paths for both studies are missing:
	/host/verges/tank/data/daniel/3T7T/z/outputs/values/MICs/subcortex/MICs_feature-ADC_analysis-regional.csv
	/host/verges/tank/data/daniel/3T7T/z/outputs/values/PNI/subcortex/PNI_feature-ADC_analysis-regional.csv
[missingPth] Paths for both studies are missing:
	/host/verges/tank/data/daniel/3T7T/z/outputs/values/MICs/subcortex/MICs_feature-T1map_analysis-regional.csv
	/host/verges/tank/data/daniel/3T7T/z/outputs/values/PNI/subcortex/PNI_feature-T1map_analysis-regional.csv
[missingPth] Paths for both studies are missing:
	/host/verges/tank/data/daniel/3T7T/z/outputs/values/MICs/subcortex/MICs_feature-volume_analysis-regional.csv
	/host/verges/tank/data/daniel/3T7T/z/outputs/values/PNI/subcortex/PNI_feature-volume_analysis-regional.csv
shape of files_lst (num files, num studies): 54 by 2


In [29]:
importlib.reload(vrtx)

<module 'vrtx' from '/host/verges/tank/data/daniel/3T7T/z/code/analyses/vrtx.py'>

# Group histograms
- All analysed PX vs all PNE for each file type


In [12]:
# get summary stats for each file type
df_summary = pd.DataFrame()
for lst in files_lst:
    for file in lst:
        print(os.path.basename(file))
        df = vrtx.summaryStats(file)
        df_summary = pd.concat([df_summary, df])

MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-2mm_analysis-regional.csv
PNI_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-2mm_analysis-regional.csv
MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-2mm_analysis-regional.csv
PNI_hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-2mm_analysis-regional.csv
MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-volume_smooth-2mm_analysis-regional.csv
PNI_hemi-L_surf-fsLR-32k_label-midthickness_feature-volume_smooth-2mm_analysis-regional.csv
MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-5mm_analysis-regional.csv
PNI_hemi-L_surf-fsLR-32k_label-midthickness_feature-ADC_smooth-5mm_analysis-regional.csv
MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-5mm_analysis-regional.csv
PNI_hemi-L_surf-fsLR-32k_label-midthickness_feature-T1map_smooth-5mm_analysis-regional.csv
MICs_hemi-L_surf-fsLR-32k_label-midthickness_feature-volume_smooth-5mm_analysis-regional.cs

In [13]:
df_summary

,study,hemi,surf,label,feat,smooth,analysis,n_rows,n_cols_all,n_cols_NA,mean,std,mdn,25perc,75perc,max,min,mode
0,MICs,L,32k,midthickness,ADC,2mm,regional,32492,17,0,956.032062,4448.105167,0.472150,-0.406609,1.422128,32491.0,-28.457537,-0.682098
0,PNI,L,32k,midthickness,ADC,2mm,regional,32492,17,2,1084.573277,4720.552236,0.780898,-0.751150,3.250555,32491.0,-240.846470,-0.577350
0,MICs,L,32k,midthickness,T1map,2mm,regional,32492,17,0,955.221797,4448.279383,-0.312292,-1.322470,0.841047,32491.0,-117.541930,-2.253015
0,PNI,L,32k,midthickness,T1map,2mm,regional,32492,17,2,1082.715660,4720.944171,-0.165786,-0.908385,0.625065,32491.0,-36.722248,-1.554285
0,MICs,L,32k,midthickness,volume,2mm,regional,32492,17,0,955.671432,4448.185684,0.009430,-0.780094,1.052932,32491.0,-25.023254,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,PNI,R,0p5mm,midthickness,ADC,5mm,regional,7262,17,2,242.176567,1055.005416,0.525727,-1.235455,1.721091,7261.0,-5.032597,-3.421859
0,MICs,R,0p5mm,midthickness,T1map,5mm,regional,7262,17,0,212.432464,994.342872,-0.961158,-2.205271,0.287986,7261.0,-28.881132,-7.550509
0,PNI,R,0p5mm,midthickness,T1map,5mm,regional,7262,17,2,241.400205,1055.183505,-0.457885,-1.553483,0.833664,7261.0,-7.110971,-6.410366
0,MICs,R,0p5mm,midthickness,volume,5mm,regional,7262,17,0,213.027661,994.212863,-0.562389,-1.163143,0.139105,7261.0,-6.404078,-2.252309


In [24]:
out_pth = os.path.join(processed_output_dir, f"sumStats_{date}.csv")
df_summary.to_csv(out_pth, index=False)
print  (f"Summary stats saved to {out_pth}")

NameError: name 'processed_output_dir' is not defined

In [ ]:
# Ridge plots (one line per participant)
# plot histogram



# check that file exists
# if not, continue to next file
for file in files_lst:
        
        
        # read in data
        df_mics = pd.read_csv(mics_file, index_col=False)
        df_pni = pd.read_csv(pni_file, index_col=False)
        # remove participants with Na in either df
        # df_mics = df_mics.dropna()
        # df_pni = df_pni.dropna()
        
        # keep only overlapping participants both dfs
        
        ## need to remap col names according to 3T-7T ID correspondence

        ## keep only overlapping columns
        # cols = df_mics.columns.intersection(df_pni.columns)
        # df_mics = df_mics[cols]
        # df_pni = df_pni[cols]

        ## Take histogram for each participant

        # print(df_mics.head())
        break
        # # construct histogram
        # fig = plots.ridge(df_mics, matrix_df = df_mics)
        # # show histogram
        # fig.show()

In [ ]:

        print(path)
        
        df = pd.read_csv(path)
        fig = plots.histStack(df)
        # display plot
        fig.show()